In [ ]:
import os
import pandas as pd
from time import time, sleep

start_time = time()

# database configuration
user = 'root'
password = '' 
host = '127.0.0.1' 
database = 'eegsignal' 
table = 'signals'

import mysql.connector

try:
    # Establish a connection to the MySQL server
    cnx = mysql.connector.connect(user=user, password=password,
                                host=host, database=database)

    if cnx.is_connected():
        cursor = cnx.cursor()
        print('Connected to MySQL database')

except Exception as e:
    print(e)

# get all the data we need
cursor.execute("SELECT * FROM signals WHERE subset IN ('B', 'C')")

# fetch the results 
results = cursor.fetchall()

time_to_fetch_data = time() - start_time

print(f"Elapsed time to fetch from db: {time_to_fetch_data} seconds.")

# create a dataframe from the results
datas = pd.DataFrame(results, columns=cursor.column_names)

datas['subset'] = datas['subset'].replace("B", 0)
datas['subset'] = datas['subset'].replace("C", 1)

data_b = datas.loc[datas['subset'] == 0]
data_b = data_b.drop(['subset', 'id', 'state'], axis=1).values.transpose()
data_b = pd.DataFrame(data_b)
data_c = datas.loc[datas['subset'] == 1]
data_c = data_c.drop(['subset', 'id', 'state'], axis=1).values.transpose()
data_c = pd.DataFrame(data_c)

datas = pd.concat([data_b, data_c])

y_true = [0] * 100 + [1] * 100

datas.head()

In [ ]:
import pywt
import numpy as np
import warnings
import nolds
import pyeeg

def create_mat(df):
    matx = np.zeros((len(df), len(df.columns)))
    for i, col_name in enumerate(df.columns):
        matx[:, i] = df[col_name].values
        sleep(0.01)
    return matx

matA = create_mat(datas)
matA = np.nan_to_num(matA)

feature_size = 4
columns_name = list()
for i in range(feature_size):
    columns_name = columns_name + ['f'+str(i+1)]

def features(mat):
    Kmax = 5
    Tau  = 4
    DE   = 10
    M    = 10
    R    = 0.3
    Band = np.arange(1,86)
    Fs   = 173
    DFA                = pyeeg.dfa(mat)
    HFD                = pyeeg.hfd(mat,Kmax)
    SVD_Entropy        = pyeeg.svd_entropy(mat,Tau,DE)
    Fisher_Information = pyeeg.fisher_info(mat,Tau,DE)
    p , p_ratio = pyeeg.bin_power(mat, Band, Fs)
    Spectral_Entropy   = pyeeg.spectral_entropy(mat, Band, Fs, Power_Ratio=p_ratio)
    PFD                = pyeeg.pfd(mat)
    mean               = np.mean(mat)
    sleep(0.01)
    return(DFA, HFD, SVD_Entropy , Spectral_Entropy, Fisher_Information, PFD)

f1_A = np.zeros((matA.shape[0],1))
f2_A = np.zeros((matA.shape[0],1))
f3_A = np.zeros((matA.shape[0],1))
f4_A = np.zeros((matA.shape[0],1))
f5_A = np.zeros((matA.shape[0],1))
f6_A = np.zeros((matA.shape[0],1))
f7_A = np.zeros((matA.shape[0],1))
cl_A = np.zeros((matA.shape[0],1))

for i in range(matA.shape[0]):
    [f1_A[i,0],f2_A[i,0],f3_A[i,0],f4_A[i,0],f5_A[i,0],f6_A[i,0]]=features(matA[i])

MftA = np.concatenate([f1_A,f2_A,f3_A,f4_A,f5_A, f6_A], axis=1)
FCM_A = pd.DataFrame(MftA,columns=['f1','f2','f3','f4','f5', 'f6'])

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X = np.asarray(FCM_A)

sc = StandardScaler()
input_data = sc.fit_transform(X)

In [ ]:
FCM_A


In [ ]:
import joblib

# load the model
loaded_dwt_model = joblib.load("decision_tree_original_values.joblib")

y_pred = loaded_dwt_model.predict(input_data)

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# Calculate the F1-score
f1 = f1_score(y_true, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_true, y_pred)
print("Precision:", precision)

# Calculate the recall score
recall = recall_score(y_true, y_pred)
print("Recall: {:.2f}".format(recall))

# Print the F1-score
print("F1-score: {:.2f}".format(f1))

print(f"Total time: {time() - start_time} seconds")

import seaborn as sns
import matplotlib.pyplot as plt     
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_true, y_pred)

sns.heatmap(conf_mat, annot=True, cmap="YlGnBu", fmt="d")

plt.title("Confusion Matrix Original Values Method")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()
